In [366]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics.pairwise import haversine_distances

In [355]:


df = pd.read_excel("Canadian Radio Playlist_V.21.xlsx", "Campus Radio Charts")
df


/home/owen/Programs/anaconda3/envs/lighthouse/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,WEEK OF,STATION,STATION CITY,STATION PROVINCE,STATION LATITUDE,STATION LONGITUDE,CHART POSITION,ARTIST NAME(S),ARTIST COUNTRY,ARTIST HOME CITY,...,LABEL TYPE,LANGUAGE OF MUSIC,VISIBLE ETHNIC MINORITY,CENSUS RACE CLASSIFICATION,ARTIST GENDER,M-MUSIC,A-ARTIST,P-PERFORMANCE,L-LYRICS,Unnamed: 24
0,2006-01-10 00:00:00,CJSR,Edmonton,AB,53.55,-113.50,4,Breakestra,US,"Los Angeles, CA",...,Indie,English,Yes,Mixed Group,Male Group,No,No,No,No,NaN
1,2006-01-10 00:00:00,CJSR,Edmonton,AB,53.55,-113.50,5,Candy's .22,US,"Los Angeles, CA",...,Indie,English,No,White,Male Group,No,No,No,No,NaN
2,2006-01-10 00:00:00,CJSR,Edmonton,AB,53.55,-113.50,7,Dangerdoom,US,"New York, NY",...,Indie,English,Yes,Black,Male,No,No,No,No,NaN
3,2006-01-10 00:00:00,CJSR,Edmonton,AB,53.55,-113.50,8,Blockhead,US,"New York, NY",...,Indie,English,No,White,Male,No,No,No,No,NaN
4,2006-01-10 00:00:00,CJSR,Edmonton,AB,53.55,-113.50,9,Blackalicious,US,"Sacramento, CA",...,Indie,English,Yes,Black,Male Group,No,No,No,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35854,3/2/1010,CKUT,Montreal,QC,45.50,-73.58,4,Kae Sun,Cdn,"Toronto, ON",...,Indie,English,Yes,Black,Male,No,No,No,No,NaN
35855,3/2/1010,CKUT,Montreal,QC,45.50,-73.58,5,Rel!g!on,Cdn,"Vancouver, BC",...,Indie,English,Yes,Black,Male,Yes,Yes,Yes,Yes,NaN
35856,3/2/1010,CKUT,Montreal,QC,45.50,-73.58,6,Swollen Members,Cdn,"Vancouver, BC",...,Indie,English,Yes,Mixed Group,Male Group,Yes,Yes,Yes,Yes,NaN
35857,3/2/1010,CKUT,Montreal,QC,45.50,-73.58,7,Chokeules,Cdn,"Toronto, ON",...,Indie,English,No,White,Male,Yes,Yes,Yes,Yes,NaN


In [296]:
df.nunique()

WEEK OF                          267
STATION                           26
STATION CITY                      21
STATION PROVINCE                   7
STATION LATITUDE                  21
STATION LONGITUDE                 21
CHART POSITION                    11
ARTIST NAME(S)                  2165
ARTIST COUNTRY                     9
ARTIST HOME CITY                 358
ARTIST HOME LATITUDE             304
ARTIST HOME  LONGITUDE           318
KM DISTANCE (HOME - STATION)    1777
ALBUM NAME                      2711
LABEL NAME                      1125
LABEL TYPE                         6
LANGUAGE OF MUSIC                 21
VISIBLE ETHNIC MINORITY            4
CENSUS RACE CLASSIFICATION        21
ARTIST GENDER                     10
M-MUSIC                            3
A-ARTIST                           3
P-PERFORMANCE                      3
L-LYRICS                           3
Unnamed: 24                        1
dtype: int64

In [297]:
df.isna().sum()[df.isna().sum() > 0]

STATION CITY                        1
STATION PROVINCE                    1
STATION LATITUDE                    1
STATION LONGITUDE                  11
CHART POSITION                     24
ARTIST NAME(S)                      1
ARTIST HOME CITY                   13
ARTIST HOME LATITUDE             1819
ARTIST HOME  LONGITUDE           1849
KM DISTANCE (HOME - STATION)     1872
ALBUM NAME                          1
LABEL NAME                          1
LABEL TYPE                          9
LANGUAGE OF MUSIC                  13
VISIBLE ETHNIC MINORITY            14
CENSUS RACE CLASSIFICATION         13
ARTIST GENDER                      13
M-MUSIC                            13
A-ARTIST                           13
P-PERFORMANCE                      13
L-LYRICS                           15
Unnamed: 24                     35850
dtype: int64

In [321]:
clean_df = df.copy()

In [319]:
def replace_nans(key:str, columns:list):
    # Create list of all given key values that have any samples with nan values in given columns
    for key_value in df[(df[columns].transpose().isna().any())][key].unique():
        #Create filters
        any_nan = (clean_df[columns].transpose().isna().any()) & (clean_df[key] == key_value)
        all_nan = (df[columns].transpose().isna().all()) & (df[key] == key_value)
        #Get values from a row that contains not nan values
        values = df[~all_nan].head(1)[columns].iloc[0]
        #Replace nans with values
        clean_df.loc[any_nan, columns] = values.tolist()

    

In [322]:
replace_nans('STATION', ['STATION CITY','STATION PROVINCE','STATION LATITUDE','STATION LONGITUDE']  )

In [324]:
clean_df.isna().sum()[clean_df.isna().sum() > 0]

CHART POSITION                     24
ARTIST NAME(S)                      1
ARTIST HOME CITY                   13
ARTIST HOME LATITUDE             1819
ARTIST HOME  LONGITUDE           1849
KM DISTANCE (HOME - STATION)     1872
ALBUM NAME                          1
LABEL NAME                          1
LABEL TYPE                          9
LANGUAGE OF MUSIC                  13
VISIBLE ETHNIC MINORITY            14
CENSUS RACE CLASSIFICATION         13
ARTIST GENDER                      13
M-MUSIC                            13
A-ARTIST                           13
P-PERFORMANCE                      13
L-LYRICS                           15
Unnamed: 24                     35850
dtype: int64

In [328]:
df[df['ARTIST NAME(S)'].isna()].values

array([[datetime.datetime(2007, 7, 17, 0, 0), 'CFUV  ', 'Victoria', 'BC',
        48.43, -123.35, 10, nan, '-', nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan]], dtype=object)

In [331]:
# Drop row with no artist data
clean_df = clean_df[~clean_df['ARTIST NAME(S)'].isna()]

In [ ]:
replace_nans('ARTIST NAME(S)', ['ARTIST HOME CITY',
                                'ARTIST HOME LATITUDE',
                                'ARTIST HOME  LONGITUDE',
                                'VISIBLE ETHNIC MINORITY', 
                                'CENSUS RACE CLASSIFICATION', 
                                'ARTIST GENDER']  )

In [333]:
clean_df.isna().sum()[clean_df.isna().sum() > 0]

CHART POSITION                     24
KM DISTANCE (HOME - STATION)     1871
LABEL TYPE                          8
LANGUAGE OF MUSIC                  12
M-MUSIC                            12
A-ARTIST                           12
P-PERFORMANCE                      12
L-LYRICS                           14
Unnamed: 24                     35849
dtype: int64

In [ ]:
replace_nans('ALBUM NAME', ['LANGUAGE OF MUSIC'])

In [337]:
replace_nans('LABEL NAME', ['LABEL TYPE'])

In [338]:
clean_df.isna().sum()[clean_df.isna().sum() > 0]

CHART POSITION                     24
KM DISTANCE (HOME - STATION)     1871
M-MUSIC                            12
A-ARTIST                           12
P-PERFORMANCE                      12
L-LYRICS                           14
Unnamed: 24                     35849
dtype: int64

In [353]:
clean_df[clean_df['CHART POSITION'].isna()].head(1)

,WEEK OF,STATION,STATION CITY,STATION PROVINCE,STATION LATITUDE,STATION LONGITUDE,CHART POSITION,ARTIST NAME(S),ARTIST COUNTRY,ARTIST HOME CITY,...,LABEL TYPE,LANGUAGE OF MUSIC,VISIBLE ETHNIC MINORITY,CENSUS RACE CLASSIFICATION,ARTIST GENDER,M-MUSIC,A-ARTIST,P-PERFORMANCE,L-LYRICS,Unnamed: 24
552,2006-02-07 00:00:00,CJSW,Calgary,AB,51.08,-114.08,NaN,Psyche Origami,US,"Atlanta, GA",...,Indie,English,Yes,Mixed Group,Male Group,No,No,No,No,NaN


In [352]:
clean_df[clean_df['ARTIST NAME(S)'] == 'Psyche Origami ' ][['CHART POSITION', 'ALBUM NAME']]

,CHART POSITION,ALBUM NAME
7,5,The Standard
13,5,The Standard
85,6,The Standard
130,1,The Standard
137,4,The Standard
184,9,The Standard
202,10,The Standard
211,8,The Standard
267,5,The Standard
274,7,The Standard


Unable to account for null chart positions due to variation within a given week / album pairing and lack of track name

In [406]:
clean_df.to_csv('clean_data.csv', index=0)